# Optimizers

> This module contains the optimizers

In [ ]:
#| export
import numpy as np
from copy import deepcopy

In [ ]:
#|default_exp optimizers

In [ ]:
#| export
def gd(x:np.ndarray, # x data of N elements
       y:np.ndarray, # y data of N elements
       pini:dict, # initial parameters of the loss function
       ll:dict, # dictionary with the loss ('loss'), the gradients ('grads') and the function ('fun') for the regression/classification
       niter=int(1E3), #number of iterations
       eta=1E-3, #learning rate
      )->dict:#dictionary containing the vector of the losses ('loss') and the parameters (following the keys of pini)
    
    loss, grads, fun = ll['loss'] , ll['grads'], ll['fun']
    
    trackers = {}
    trackers['loss'] = [loss(x,y,fun,pini)]
    for p in pini: trackers[p] = [pini[p]]
    
    params = deepcopy(pini)
    for i in range(niter):
        g= grads(x,y,params)
        trackers['loss'] = trackers['loss']+ [loss(x,y,fun,params)]
        for i,p in enumerate(params): 
            trackers[p] = trackers[p]+ [trackers[p][-1]-eta*g[i]]
            params[p] = trackers[p][-1]
    return trackers

In [ ]:
#| export
def sgd_epoch(x:np.ndarray, # x data of N elements
              y:np.ndarray, # y data of N elements
              pini:dict, # initial parameters of the loss function
              ll:dict, # dictionary with the loss ('loss'), the gradients ('grads') and the function ('fun') for the regression/classification
              bs=10, #Batch size
              eta=1E-3 #learning rate
             )->dict:#dictionary containing the vector of the losses ('loss') and the parameters (following the keys of pini) 
    mask = np.arange(x.size)
    np.random.shuffle(mask)
    nb = x.size//bs
    lb = np.mod(x.size,bs)
    
    loss, grads, fun = ll['loss'] , ll['grads'], ll['fun']
    
    trackers = {}
    trackers['loss'] = [loss(x,y,fun,pini)]
    for p in pini: trackers[p] = [pini[p]]

    params = deepcopy(pini)
    for i in range(nb):
        m = mask[i*bs:(i+1)*bs]
        xx, yy = x[m], y[m]
        g= grads(xx,yy,params)
        trackers['loss'] = trackers['loss']+ [loss(x,y,fun,params)]
        for i,p in enumerate(params): 
            trackers[p] = trackers[p]+ [trackers[p][-1]-eta*g[i]]
            params[p] = trackers[p][-1]
    return trackers

In [ ]:
#| export
def sgd(x:np.ndarray, # x data of N elements
        y:np.ndarray, # y data of N elements
        pini:dict, # initial parameters of the loss function
        ll:dict, # dictionary with the loss ('loss'), the gradients ('grads') and the function ('fun') for the regression/classification
        bs=10, #Batch size
        eta=1E-3, #learning rate
        niter=int(1E3) #number of epochs
        )->dict:#dictionary containing the vector of the losses ('loss') and the parameters (following the keys of pini) 
    params = deepcopy(pini)
    for i in range(niter):
        track = sgd_epoch(x, y, params, ll, bs=bs, eta=eta)
        if i==0: 
            trackers = track
        else:
            for i,p in enumerate(trackers): trackers[p] = trackers[p]+ track[p]
        for i,p in enumerate(params): params[p] = trackers[p][-1]
    return trackers

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()